---

## 8.1 Experimentación

Se realizaron, para aquellos modelos con mejores resultados en la fase de gridsearch, una validación cruzada. Esta consistió en el uso del método CrossValidation de sklearn, lo cual se realizó con el conjunto entero de datos.

Recordemos que el método particiona el conjunto en 5 partes (valor por defecto que mantuvimos). Entrena el modelo con 4 partes de 5 y testea con el quinto restante. Esto genera cinco resultados diferentes, que es lo que analizamos.

Hemos así testeado los métodos por separado, pero también agregamos un estimador de voto en casa caso, reuniendo el poder de predicción de cada uno de los métodos utilizados. Esto resultó de importancia para los buenos resultados que obtuvimos.

### Experimentos para clasificación

In [1]:
from sklearn.model_selection import train_test_split, cross_val_score
import pandas as pd

import sys
import os
project_path = os.path.abspath('..')
sys.path.insert(1, project_path)

from tempfile import mkdtemp
from sklearn.base import BaseEstimator, TransformerMixin
from preprocessing import Nothing, CategoriesTokenizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import MinMaxScaler, PowerTransformer, OneHotEncoder
from sklearn.feature_selection import SelectPercentile, f_classif
from sklearn.pipeline import Pipeline
import re

from src.features.preprocessing import boc_many_values, boc_some_values, Nothing, CategoriesTokenizer, custom_features, preprocessing_bert

In [2]:
from transformers import AutoModelForSequenceClassification
from transformers import AutoTokenizer
import numpy as np

MODEL = "distilbert-videogame-descriptions-rating"

tokenizer = AutoTokenizer.from_pretrained(MODEL)
model = AutoModelForSequenceClassification.from_pretrained(MODEL)

def sentence_clf_output(text):
    """retorna el SequenceClassifierOutput"""
    encoded_input = tokenizer(text, return_tensors='pt')
    output = model(**encoded_input, return_dict=True, output_hidden_states=True)
    return output

def logits_embedding(clf_output):
    # retorna el vector de scores de clasificacion (antes de la capa softmax)
    return clf_output['logits'][0].detach().numpy().reshape(1,5)

In [3]:
def integrar_bert_logits(df_in):
    df = df_in.copy(deep=True)

    embed = lambda row: logits_embedding(sentence_clf_output(row))
    bert_logits = np.concatenate(df['short_description'].apply(embed).to_numpy())  # .reshape(100,3)

    df[['bert1','bert2','bert3','bert4','bert5']] = pd.DataFrame(bert_logits, index= df.index)

    return df

In [4]:
df_train = pd.read_pickle('train.pickle')
df_train = integrar_bert_logits(df_train)

In [6]:
df_train = custom_features(df_train)
# X_train, X_eval, y_train, y_eval = train_test_split(df_train, df_train['rating'], test_size=0.3, random_state=0, stratify=df_train['rating'])

In [7]:
def make_pipeline(clf):
    pipeline = Pipeline(
        [("procesamiento", preprocessing_bert),
        ("selector", SelectPercentile(f_classif, percentile=95)),
        ("classifier", clf)]
    )
    return pipeline

### Borrador de GridSearch (clasificación)

**Candidatos**:
- SVCLineal
- KNeighbors
- RandomForest
- MLP

In [8]:
from sklearn.svm import LinearSVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier, VotingClassifier
from sklearn.neural_network import MLPClassifier

In [10]:
clasificadores_exp = [
    LinearSVC(random_state=0),
    KNeighborsClassifier(weights='distance'),
    RandomForestClassifier(n_estimators=250, random_state=0),
    MLPClassifier(hidden_layer_sizes=(200,), learning_rate_init=0.01, solver='sgd',random_state = 0),
    VotingClassifier(estimators=[
        ('SVC', LinearSVC(random_state=0)),
        ('KN', KNeighborsClassifier(weights='distance')),
        ('RF', RandomForestClassifier(n_estimators=250, random_state=0)),
        ('MLP', MLPClassifier(hidden_layer_sizes=(200,), learning_rate_init=0.01, solver='sgd',random_state = 0))
    ])
]

In [11]:
%%capture
results_cv = {}

for classif in clasificadores_exp:
    pipe = make_pipeline(reg)
    score = cross_val_score(pipe,df_train, df_train['rating'],scoring='f1_weighted')
    results_cv[type(reg).__name__] = score
    # print("CV SCORE {}: {}".format(type(reg).__name__,score))

In [12]:
for classif in results_cv.keys():
    print("CV SCORE {}: {}".format(classif,results_cv[classif]))

CV SCORE LinearSVC: [0.36057687 0.34802132 0.37176183 0.35326648 0.34257792]
CV SCORE KNeighborsClassifier: [0.29278178 0.28670371 0.31304605 0.31712508 0.30444146]
CV SCORE RandomForestClassifier: [0.35857368 0.36460172 0.37187043 0.35138941 0.35626242]
CV SCORE MLPClassifier: [0.36815039 0.35760368 0.37994783 0.37399761 0.35189534]
CV SCORE VotingClassifier: [0.36662592 0.36160182 0.37349064 0.37553451 0.35211995]


| Modelo | Fold 1 | Fold 2 | Fold 3 | Fold 4 | Fold 5 | **Promedio** |
|:---:|:---:|:---:|:---:|:---:|:---:|:---:|
| **LinearSVC** | 0.36057687 | 0.34802132 | 0.37176183 | 0.35326648 | 0.34257792 | **0.3551** |
| **KNeighborsClassifier** | 0.29278178 | 0.28670371 | 0.31304605 | 0.31712508 | 0.30444146 | **0.3027** |
| **RandomForestClassifier** | 0.35857368 | 0.36460172 | 0.37187043 | 0.35138941 | 0.35626242 | **0.3604** |
| **MLPClassifier** | 0.36815039 | 0.35760368 | 0.37994783 | 0.37399761 | 0.35189534 | **0.3662** |
| **VotingClassifier** | 0.36662592 | 0.36160182 | 0.37349064 | 0.37553451 | 0.35211995 | **0.3658** |

Los métodos muestran una relativa estabilidad en los distintos folds. Esto nos asegura la estabilidad de nuestros modelos. El modelo con mejor resultado fue el perceptrón multi-capa, con una ganancia marginal por sobre el estimador de voto al comparar el promedio de los resultados. Sin embargo, al utilizar este estimador para la predicción en codalab se obtuvo un punto porcentual menos de score f1 con respecto al voto de varios modelos.